## Uncovering Social Media Trends With ETPOL
### COSC-247 Machine Learning: Application Final Project Writeup
### Fynn Hayton-Ruffner

### Overview

Identifying political bias has always been a topic of great fascination for machine learning practicioners. Though this topic spans a wide breadth of distinct application areas, particular attention is paid to bias in news articles (CITE). This is not without good reason; identifying bias is a skill essential to any education that values critical thinking. While the value of article classififcation is undeniable, a gap in the modern literature exists for working with the political ideology of social media posts. In the past, many simpler, though remarkably effective methods were applied to identify the ideology of twitter posts (CITE). These studies used Decision Trees and Support Vector Machines, which, while very effective tools, can only make sense of sequential textual data as a bag of words. THIS STUDY, worked with youtube titles...While this study used transformeres. Though this is at its core an application project, I was interested to see how well the transformer model I created (which I'm calling ETPOL), measured up against these modern benchmarks. I was not expecting to outperform Google's BERT model due to an incomprehensible gap in skill, resources, and time, but I was looking for something in the range of 82 - 90%. The bottom of this range just exceeds the most stellar performances from the standard machine learning models that I could find in less recent, prior research. I was hopeful that ETPOL could beat those models due to the transformer's exceptional ability to handle sequential data, but admittedly a little worried that a dataset on the smaller end of things (~150,000) might hold it back. Nonetheless, 82-90% was my designated range of sucess, which I managed to achieve. I then applied the resulting model to the TWITTER THREADS....In this writeup, I begin by describing the data and ETPOL, after which I detail the entire training process and resulting model, followed by its application to the twitter threads of interest, and finishing with a discussion of conclusions, implications, and future work.

##### The Data
To start, I had to find a sizeable number of social media posts labeled by political inclination. Luckily, I found two datasets on kaggle that had just the thing: [Messy Twitter Dataset](https://www.kaggle.com/datasets/cascathegreat/politicaltweets) with ~130,000 posts, and [Clean Reddit Dataset](https://www.kaggle.com/datasets/neelgajare/liberals-vs-conservatives-on-reddit-13000-posts/data) with 13,000 posts. However, while I was lucky to find these datasets, they were structured completely differently, so combining them into one took a lot of cleaning and reformatting. I have the full cleaning process in 'clean_data.ipynb' but to sum up, I reformatted the post content of the messy dataset (which was such a pain), removed everything that wasn't alphanumeric or punctuation with regex, renamed columns for consistency, and dealt with invisible unicode chars. When I hopefully use the resulting model on twitter threads, I will have to do apply this same cleaning process to that data for consistency. Nonetheless, after all that, I was able to join the two datasets together to create an 'all_posts.csv' file with 146,478 rows and two columns: 'content' & 'affiliation' (with values left or right); this size is far greater than anything I worked with this semester in this class but it is still relatively small in the grand scheme of ML. For reference, both the polished dataset and the raw csv files are in the 'data' directory of this project.

In [1]:
import pandas as pd
from pathlib import Path

df=pd.read_csv(Path('data/all_posts.csv'))
df.head()

,content,affiliation
0,when you look at the history of big social mov...,left
1,it was great to be back in new jersey! there's...,left
2,"virginians delivered for me twice, and now im ...",left
3,some of the most important changes often start...,left
4,glad i had a chance to talk with our new champ...,left


##### The Tokenizer
After preprocessing the data, the next step in text analysis is to create a tokenizer that maps text to the numerical format necessary to pass it through a transformer. I document the process in 'tokenizer.py' but will summarize it here. To start, I created a word piece tokenizer following the tutorial on [huggingface.com](https://huggingface.co/learn/llm-course/en/chapter6/8). Word piece is a subword tokenizer that breaks words into meaningful sub-units. For example, 'modernization' may become 'modern' & 'ization' since 'ization' may be encountered as a common suffix in the vocabulary. This tackles the common issue of unknown words in new data. Since the tokenizer always starts from every character encountered in the vocab, as long as the input text doesn't have any new characters it will be able to handle it. Given the size of the dataset and that I formatted the input data to only include alphanumeric chars and punctuation, it is very unlikely for the tokenizer to encounter characters not already in its vocabularly. After training the tokenizer on my dataset in tokenizer.py, I its state in the 'wp_tokenizer' directory. It has a 20,000 token vocabulary, which was more than enough to account at least for all the unique characters in my dataset. I provide an example below of how the tokenizer encodes a string into its numerical representation and back again. You'll notice if you run it that not only is the decoded text string normalized (to lower case and uniform whitespace separation), but a new [CLS] token appears at the front. It is standard practice in text classification tasks to include this special CLS token as the token from which the class probabilities of the entire text are extracted (CITE BERT).

In [3]:
from transformers import PreTrainedTokenizerFast
from pathlib import Path 

test_str = "I am a test string!"
tokenizer = PreTrainedTokenizerFast.from_pretrained(Path('wp_tokenizer'), model_max_length=25)

tokenized = tokenizer.encode(test_str)
print(f'Tokens: {tokenized}')
print(f'Decoded tokens: {tokenizer.decode(tokenized)}')

Tokens: [2, 33, 157, 25, 1530, 19216, 3]
Decoded tokens: [CLS] i am a test string !


##### ETPOL
As mentioned above, I decided to try to code my own encoder model using the pytorch module, since transformers tend to outperform other machine learning model on text related tasks. The reason for this gap is attention, the key process of a transformer that enriches the meaning of every token by the context that surrounds it (kind of like how we interpret text). To create the model, I followed the structure of an encoder from the [Attention is All You Need](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) paper, which introduced transformers to the world. The component flow for any input sequence X is (writing this out always helps me better understand what this does):

X shape: (512) -> 512 tokens in input sequence

1: embedding layer: this converts all tokens to an embedding vector of size 'd_model'. Each word or subword becomes a long sequence of torch.longs that encodes its initial meaning before attention and learning.

X shape: (512 x 512): now every token is represented by an embedding vector of dimension 512

2: Positional Encoder: I followed the math directly from the paper. The positional encoder is a context_size x d_model matrix that holds a vector containing embeddings for every possible position in the input sequence. When the input passes through here, the positional embeddings are added to the corresponding input embeddings with the hope of adding positional information to each token's represenation.

X shape: (512 x 512): still the same, each token embedding is just updated by the positional encoder.

3: Encoder block:

This layer holds the bulk of the work. The token embeddings are first split across multiple attention heads, each assigned to different sub components of a token's meaning, and then updated by the embeddings of all other tokens in the sequence. The embeddings that result from attention are then added back to the inital token embeddings and passed through a standard multilayer perceptron network. I'm leaving out layer normalization and dropout which are methods to help with training and reducing overfitting because the bulk of the job is done with the attention -> feedforward flow, encapsulating one encoder block. Transformers typically stack encoder blocks to add as much information to the original tokenized inputs as possible. I incorporate the number of encoder blocks as a hyperparameter to my model 'num_encoders'. 

X shape: (512 x 512), the token embeddings are now richly encoded by the context provided by their surrounding neighbors.

4: Linear output: after being passed through the linear output, I extract the [CLS] token, which is inserted at the beginning of every input sequence. This token is meant to contain the label for classfication tasks after sufficient training, and is a common method for classfication in transformers. I then extract the class probablities it assigns to liberal and conservative, and use that to update all the parameters of the model.

X shape (2): just raw logit probs for each class

Below is an example pass through the model using a random observation in the dataset.